In [1]:
"""
@my_dec
def my_func():
    pass

my_func = my_dec(my_func)

#######

@my_dec
class MyClass:
    pass

MyClass = my_dec(MyClass)
"""


'\n@my_dec\ndef my_func():\n    pass\n\nmy_func = my_dec(my_func)\n\n#######\n\n@my_dec\nclass MyClass:\n    pass\n\nMyClass = my_dec(MyClass)\n'

In [2]:
def savings(cls):
    # class decorator example
    cls.account_type = "savings"
    return cls

def checking(cls):
    cls.account_type = "checking"
    return cls


class Account:
    pass


@savings
class Bank1Savings(Account):
    pass


@savings
class Bank2Savings(Account):
    pass


@checking
class Bank1Checking(Account):
    pass


@checking
class Bank2Checking(Account):
    pass

In [3]:
Bank1Savings.__dict__, Bank2Savings.__dict__

(mappingproxy({'__module__': '__main__',
               '__doc__': None,
               'account_type': 'savings'}),
 mappingproxy({'__module__': '__main__',
               '__doc__': None,
               'account_type': 'savings'}))

In [4]:
Bank1Checking.__dict__, Bank2Checking.__dict__

(mappingproxy({'__module__': '__main__',
               '__doc__': None,
               'account_type': 'checking'}),
 mappingproxy({'__module__': '__main__',
               '__doc__': None,
               'account_type': 'checking'}))

In [5]:
def account_type(account_type):    
    def decorator(cls):
        cls.account_type = account_type
        return cls
    return decorator


In [6]:
@account_type("savings")
class Bank1Savings:
    pass


@account_type("checking")
class Bank1Checking:
    pass


In [7]:
Bank1Savings.__dict__, Bank1Checking.__dict__

(mappingproxy({'__module__': '__main__',
               '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
               '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
               '__doc__': None,
               'account_type': 'savings'}),
 mappingproxy({'__module__': '__main__',
               '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
               '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
               '__doc__': None,
               'account_type': 'checking'}))

In [8]:
def hello(cls):
    cls.hello = lambda self: f"Hello from {self}!"
    return cls


@hello
class Person:
    pass


Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [9]:
p = Person()
type(p.hello), p.hello()

(method, 'Hello from <__main__.Person object at 0x10f089550>!')

In [10]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"Log: {fn.__qualname__}({args}, {kwargs}) = {result}")
        return result
    return inner


In [11]:
class Person:

    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @func_logger
    def greet(self):
        return f"Hi, my name is {self.name} and my age is {self.age} years old."


In [12]:
p = Person("Bob", 33)


Log: Person.__init__((<__main__.Person object at 0x10f08acc0>, 'Bob', 33), {}) = None


In [13]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x10f08acc0>,), {}) = Hi, my name is Bob and my age is 33 years old.


'Hi, my name is Bob and my age is 33 years old.'

In [14]:
def class_logger(cls):
    # won't work well with static and class methods!
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating", cls, name, obj)
            # decorate each callable with func_logger
            setattr(cls, name, func_logger(obj))
    return cls

In [15]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f"Hi, my name is {self.name} and my age is {self.age} years old." 

decorating <class '__main__.Person'> __init__ <function Person.__init__ at 0x10f07f880>
decorating <class '__main__.Person'> greet <function Person.greet at 0x10f07f920>


In [16]:
p = Person("John", 43)

Log: Person.__init__((<__main__.Person object at 0x10f0a8170>, 'John', 43), {}) = None


In [17]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x10f0a8170>,), {}) = Hi, my name is John and my age is 43 years old.


'Hi, my name is John and my age is 43 years old.'

In [18]:
@class_logger
class Person:
    # cls_method is a descriptor, not a callable! Our `class_logger` looks for callables
    
    @staticmethod
    def static_method():
        print("statis_method invoked")

    @classmethod
    def cls_method(cls):
        print(f"cls_method invoked for {cls}...")

    def instance_method(self):
        print(f"instance_method invoked for {self}...")


decorating <class '__main__.Person'> static_method <staticmethod(<function Person.static_method at 0x10f0ac400>)>
decorating <class '__main__.Person'> instance_method <function Person.instance_method at 0x10f0ac540>


In [19]:
p = Person()
try:
    # doesn't work - throws an error
    p.static_method()
except TypeError as e:
    print(e)

Person.static_method() takes 0 positional arguments but 1 was given


In [20]:
p.cls_method() # wasn't even decorated

cls_method invoked for <class '__main__.Person'>...


In [21]:
p.instance_method()

instance_method invoked for <__main__.Person object at 0x10edb2630>...
Log: Person.instance_method((<__main__.Person object at 0x10edb2630>,), {}) = None


In [22]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'static_method': <function __main__.Person.static_method()>,
              'cls_method': <classmethod(<function Person.cls_method at 0x10f0ac4a0>)>,
              'instance_method': <function __main__.Person.instance_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [23]:
Person.__dict__["static_method"], callable(Person.__dict__["static_method"])  # is a callable but doesn't expect any attrs

(<function __main__.Person.static_method()>, True)

In [24]:
Person.__dict__["cls_method"], callable(Person.__dict__["cls_method"])

(<classmethod(<function Person.cls_method at 0x10f0ac4a0>)>, False)

In [25]:
class Person:
    @func_logger
    @staticmethod
    def static_method(*args, **kwargs):  # works if we allow *args, **kwargs
        print(f"static_method called with: {args}, {kwargs}")

In [26]:
p = Person()

In [27]:
p.static_method()

static_method called with: (<__main__.Person object at 0x10f0aac30>,), {}
Log: Person.static_method((<__main__.Person object at 0x10f0aac30>,), {}) = None


In [28]:
class Person:
    @staticmethod
    @func_logger
    def static_method():  # works if we change the decorators order
        print("static_method called")


In [29]:
p = Person()
p.static_method()

static_method called
Log: Person.static_method((), {}) = None


In [30]:
class Person:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def cls_method(cls):
        pass

# it's possible to access the orginal function
Person.__dict__["static_method"].__func__

<function __main__.Person.static_method()>

In [31]:
def class_logger(cls):
    # improved version that handles static and class methods
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod):
            print("decorating staticmethod", cls, name, obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            static_method = staticmethod(decorated_func)
            setattr(cls, name, static_method)
        elif isinstance(obj, classmethod):
            print("decorating classmethod", cls, name, obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            class_method = classmethod(decorated_func)
            setattr(cls, name, class_method)
        elif callable(obj):
            print("decorating regular callable", cls, name, obj)
            # decorate each callable with func_logger
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls


In [32]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print("static_method called...")

    @classmethod
    def cls_method(cls):
        print("cls_method called...")

    def instance_method(self):
        print("instance_method called...")
   

decorating staticmethod <class '__main__.Person'> static_method <staticmethod(<function Person.static_method at 0x10f0af420>)>
decorating classmethod <class '__main__.Person'> cls_method <classmethod(<function Person.cls_method at 0x10f0af4c0>)>
decorating regular callable <class '__main__.Person'> instance_method <function Person.instance_method at 0x10f0af560>


In [33]:
Person.static_method()

static_method called...
Log: Person.static_method((), {}) = None


In [34]:
Person.cls_method()

cls_method called...
Log: Person.cls_method((<class '__main__.Person'>,), {}) = None


In [35]:
p = Person()
p.instance_method()

instance_method called...
Log: Person.instance_method((<__main__.Person object at 0x10f0bdcd0>,), {}) = None


In [36]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name

    # property still not handled as required - decorator isn't used on it
    @property
    def name(self):
        return self._name


decorating regular callable <class '__main__.Person'> __init__ <function Person.__init__ at 0x10f0d04a0>


In [37]:
type(Person.__dict__["name"])

property

In [38]:
isinstance(Person.__dict__["name"], property)

True

In [39]:
prop = Person.__dict__["name"]

In [40]:
prop.fget, prop.fset, prop.fdel

(<function __main__.Person.name(self)>, None, None)

In [41]:
def class_logger(cls):
    # improved version that handles static and class methods
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod):
            print("decorating staticmethod", cls, name, obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            static_method = staticmethod(decorated_func)
            setattr(cls, name, static_method)
        elif isinstance(obj, classmethod):
            print("decorating classmethod", cls, name, obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            class_method = classmethod(decorated_func)
            setattr(cls, name, class_method)
        elif isinstance(obj, property):
            print("decorating property", cls, name, obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif callable(obj):
            print("decorating regular callable", cls, name, obj)
            # decorate each callable with func_logger
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
            
    return cls

In [42]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        return self._name


decorating regular callable <class '__main__.Person'> __init__ <function Person.__init__ at 0x10f0ad6c0>
decorating property <class '__main__.Person'> name <property object at 0x10f06c3b0>


In [43]:
p = Person("Bob")
p.name

Log: Person.__init__((<__main__.Person object at 0x10f0ab950>, 'Bob'), {}) = None
Log: Person.name((<__main__.Person object at 0x10f0ab950>,), {}) = Bob


'Bob'

In [45]:
# callables are not necesarly functions! They can be classes for example
@class_logger
class Person:
    class Other:
        def __call__(self):
            print("called instance of Other...")
    other = Other()

decorating regular callable <class '__main__.Person'> Other <class '__main__.Person.Other'>
decorating regular callable <class '__main__.Person'> other <__main__.Person.Other object at 0x10f0aa630>


In [47]:
Person.Other  # now it's a function, which is a problem

<function __main__.Person.Other()>

In [48]:
Person.other

<function __main__.func_logger.<locals>.inner()>

In [49]:
import inspect


class MyClass:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def cls_method(cls):
        pass

    def inst_method(self):
        pass

    @property
    def name(self):
        pass

    def __add__(self, other):
        pass

    class Other:
        def __call__(self):
            pass

    other = Other()
    

In [50]:
keys = ("static_method", "cls_method", "inst_method", "name", "__add__", "Other", "other")
inspect_funcs = ("isroutine", "ismethod", "isfunction", "isbuiltin", "ismethoddescriptor")


In [54]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format("", f"{max_fname_length}s"), "\t".join(format(key, f"{max_header_length}s") for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f"{max_header_length}s") for key in keys)
    print(format(inspect_func, f"{max_fname_length}s"), "\t".join(inspect_results))


                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


In [55]:
help(inspect.isroutine)

Help on function isroutine in module inspect:

isroutine(object)
    Return true if the object is any kind of function or method.



In [58]:
def class_logger(cls):
    # improved version that handles static and class methods
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod):
            print("decorating staticmethod", cls, name, obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            static_method = staticmethod(decorated_func)
            setattr(cls, name, static_method)
        elif isinstance(obj, classmethod):
            print("decorating classmethod", cls, name, obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            class_method = classmethod(decorated_func)
            setattr(cls, name, class_method)
        elif isinstance(obj, property):
            print("decorating property", cls, name, obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print("decorating isroutine", cls, name, obj)
            # decorate each callable with func_logger
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
            
    return cls

In [60]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def cls_method(cls):
        pass

    def inst_method(self):
        pass

    @property
    def name(self):
        pass

    def __add__(self, other):
        pass

    @class_logger
    class Other:
        def __call__(self):
            pass

    other = Other()

decorating isroutine <class '__main__.MyClass.Other'> __call__ <function MyClass.Other.__call__ at 0x10f18b100>
decorating staticmethod <class '__main__.MyClass'> static_method <staticmethod(<function MyClass.static_method at 0x10f1889a0>)>
decorating classmethod <class '__main__.MyClass'> cls_method <classmethod(<function MyClass.cls_method at 0x10f18bc40>)>
decorating isroutine <class '__main__.MyClass'> inst_method <function MyClass.inst_method at 0x10f189b20>
decorating property <class '__main__.MyClass'> name <property object at 0x10fb1e840>
decorating isroutine <class '__main__.MyClass'> __add__ <function MyClass.__add__ at 0x10f18ad40>


In [65]:
def class_logger(cls):
    # improved version that handles static and class methods
    for name, obj in vars(cls).items():

        if isinstance(obj, classmethod) or isinstance(obj, staticmethod):
            print("decorating classmethod or staticmethod", cls, name, obj)
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print("decorating property", cls, name, obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print("decorating isroutine", cls, name, obj)
            # decorate each callable with func_logger
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
            
    return cls

In [66]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def cls_method(cls):
        pass

    def inst_method(self):
        pass

    @property
    def name(self):
        pass

    def __add__(self, other):
        pass

    @class_logger
    class Other:
        def __call__(self):
            pass

    other = Other()

decorating isroutine <class '__main__.MyClass.Other'> __call__ <function MyClass.Other.__call__ at 0x10f0d05e0>
decorating classmethod or staticmethod <class '__main__.MyClass'> static_method <staticmethod(<function MyClass.static_method at 0x10f0d1760>)>
decorating classmethod or staticmethod <class '__main__.MyClass'> cls_method <classmethod(<function MyClass.cls_method at 0x10f0d1c60>)>
decorating isroutine <class '__main__.MyClass'> inst_method <function MyClass.inst_method at 0x10f0d2e80>
decorating property <class '__main__.MyClass'> name <property object at 0x10fb1e840>
decorating isroutine <class '__main__.MyClass'> __add__ <function MyClass.__add__ at 0x10f0d3ce0>


In [ ]:
def class_logger(cls):
    # improved version that handles static and class methods
    for name, obj in vars(cls).items():

        if isinstance(obj, classmethod) or isinstance(obj, staticmethod):
            print("decorating classmethod or staticmethod", cls, name, obj)
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print("decorating property", cls, name, obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print("decorating isroutine", cls, name, obj)
            # decorate each callable with func_logger
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
            
    return cls